In [1]:
###############################################################################
#define context, module with important variables
###############################################################################
from src.py21cnn import ctx
ctx.init()
###############################################################################
#parsing inputs
###############################################################################
import argparse
parser = argparse.ArgumentParser(prog = 'Large Database Model Run')

parser.add_argument('--simple_run', type=int, choices=[0, 1], default = 0)
parser.add_argument('--X_shape', type=str, default="25,25,526")
parser.add_argument('--N_walker', type=int, default=10000)
parser.add_argument('--N_slice', type=int, default=4)
parser.add_argument('--N_noise', type=int, default=10)
parser.add_argument('--noise_rolling', type=int, choices=[0, 1], default = 0)
parser.add_argument('--workers', type=int, default=24)
parser.add_argument('--verbose', type=int, choices=[0, 1, 2], default=2)
parser.add_argument('--X_fstring', type=str, default="lightcone_depthMhz_0_walker_{:04d}_slice_{:d}_seed_{:d}")
parser.add_argument('--data_fstring', type=str, default="/scratch/d.prelogovic/data/SKA_1000_tfrecord/{}_seed_{:d}_{:03d}_of_{:03d}")
parser.add_argument('--saving_location', type=str, default="/scratch/d.prelogovic/runs/models/")
parser.add_argument('--logs_location', type=str, default="/scratch/d.prelogovic/runs/logs/")
parser.add_argument('--model', type=str, default="playground.SummarySpace3D_simple")
parser.add_argument('--model_type', type=str, default="RNN")
parser.add_argument('--HyperparameterIndex', type=int, default=0)
parser.add_argument('--epochs', type=int, default=10)
parser.add_argument('--max_epochs', type=int, default=-1)
parser.add_argument('--gpus', type=int, default=1)
parser.add_argument('--LR_correction', type=int, choices=[0, 1], default=1)
parser.add_argument('--file_prefix', type=str, default="")
# parser.add_argument('--patience', type=int, default=10)
parser.add_argument('--pTVT', type=str, default = "0.8,0.1,0.1")
parser.add_argument('--warmup', type=int, default=0)
parser.add_argument('--tf', type = int, choices = [1, 2], default = 1)

inputs = parser.parse_args("")
inputs.LR_correction = bool(inputs.LR_correction)
inputs.simple_run = bool(inputs.simple_run)
inputs.noise_rolling = bool(inputs.noise_rolling)
inputs.pTVT = [float(i) for i in inputs.pTVT.split(',')]
inputs.model = inputs.model.split('.')
inputs.X_shape = tuple([int(i) for i in inputs.X_shape.split(',')])
if len(inputs.model_type) == 0:
    inputs.model_type = inputs.model[0]
if inputs.max_epochs == -1:
    inputs.max_epochs = inputs.epochs
elif inputs.max_epochs < inputs.epochs:
    raise ValueError("epochs shouldn't be larger than max_epochs")

print("INPUTS:", inputs)
ctx.inputs = inputs

INPUTS: Namespace(HyperparameterIndex=0, LR_correction=True, N_noise=10, N_slice=4, N_walker=10000, X_fstring='lightcone_depthMhz_0_walker_{:04d}_slice_{:d}_seed_{:d}', X_shape=(25, 25, 526), data_fstring='/scratch/d.prelogovic/data/SKA_1000_tfrecord/{}_seed_{:d}_{:03d}_of_{:03d}', epochs=10, file_prefix='', gpus=1, logs_location='/scratch/d.prelogovic/runs/logs/', max_epochs=10, model=['playground', 'SummarySpace3D_simple'], model_type='RNN', noise_rolling=False, pTVT=[0.8, 0.1, 0.1], saving_location='/scratch/d.prelogovic/runs/models/', simple_run=False, tf=1, verbose=2, warmup=0, workers=24)


In [2]:
###############################################################################
#seting up GPUs
###############################################################################
import tensorflow as tf
import horovod.tensorflow.keras as hvd
if ctx.inputs.tf == 1:
    # tf.compat.v1.enable_eager_execution()
    if ctx.inputs.gpus == 1:
        # #setting up GPU
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.per_process_gpu_memory_fraction = 1. #setting the percentage of GPU usage
        config.gpu_options.visible_device_list = "0" #for picking only some devices
        config.gpu_options.allow_growth = True
        # config.log_device_placement=True
#         tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
        tf.compat.v1.enable_eager_execution(config=config)
    elif ctx.inputs.gpus > 1:
        #init Horovod
        hvd.init()
        # Horovod: pin GPU to be used to process local rank (one GPU per process)
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        config.gpu_options.visible_device_list = str(hvd.local_rank())
        tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
        # tf.compat.v1.enable_eager_execution(config=config)
    else:
        raise ValueError('number of gpus shoud be > 0')
else:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    if ctx.inputs.gpus > 1:
        hvd.init()
        tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')

    # #assuming each node has one gpu, for other configurations, has to be properly modified
    # #gpus has only one member
    # gpu = gpus[0]
    # tf.config.experimental.set_memory_growth(gpu, True)
    # if ctx.inputs.gpus > 1:
    #     hvd.init()
    # tf.config.experimental.set_visible_devices(gpu, "GPU")

#importing keras at the end, I had some issues if I import it before setting GPUs
from tensorflow import keras
keras.backend.set_image_data_format('channels_last')

if ctx.inputs.gpus > 1:
    print("HVD.SIZE", hvd.size())

if ctx.inputs.gpus > 1:
    ctx.main_process = True if hvd.rank() == 0 else False
else:
    ctx.main_process = True

In [3]:
###############################################################################
#seting hyperparameters
###############################################################################
import copy
import itertools
import sys
from src.py21cnn import utilities
from src.py21cnn import hyperparameters
if ctx.inputs.simple_run == True:
    HP_dict = hyperparameters.HP_simple()
else:
    HP = hyperparameters.HP()
    HP_list = list(itertools.product(*HP.values()))
    HP_dict = dict(zip(HP.keys(), HP_list[ctx.inputs.HyperparameterIndex]))    

#correct learning rate for multigpu run
if ctx.inputs.LR_correction == True:
        HP_dict["LearningRate"] *= ctx.inputs.gpus
        
HP = utilities.AuxiliaryHyperparameters(
    model_name=f"{ctx.inputs.model[0]}_{ctx.inputs.model[1]}", 
    Epochs=ctx.inputs.epochs, 
    MaxEpochs=ctx.inputs.max_epochs, 
    NoiseRolling=ctx.inputs.noise_rolling,
    **HP_dict,
    )

print("HYPERPARAMETERS:", str(HP))
ctx.HP = HP
print(ctx.HP.hash())


HYPERPARAMETERS: Loss:mse__Optimizer:Adam__LR:0.0010000000__Activation:relu__BN:True__dropout:0.20__reduceLR:True__Batch:00020__Epochs:00010__NoiseRolling:False
c499ab7d84c5b92a5928da1a728918d3


In [4]:
def decode(serialized_example, model_type):
    """Parses an image and label from the given `serialized_example`."""
    print(serialized_example)
    example = tf.io.parse_single_example(
        serialized_example,
        features={
            'Xx': tf.io.FixedLenFeature([], tf.int64),
            'Xy': tf.io.FixedLenFeature([], tf.int64),
            'Xz': tf.io.FixedLenFeature([], tf.int64),
            'X': tf.io.FixedLenFeature([], tf.string),
            'Yx': tf.io.FixedLenFeature([], tf.int64),
            'Y': tf.io.FixedLenFeature([], tf.string),
        })
    xx = tf.cast(example['Xx'], tf.int64)
    xy = tf.cast(example['Xy'], tf.int64)
    xz = tf.cast(example['Xz'], tf.int64)
    x = tf.io.decode_raw(example['X'], tf.float32)
    x = tf.reshape(x, (xx, xy, xz))
    if model_type == "RNN":
        x = tf.transpose(x)
    x = tf.expand_dims(x, -1)
    yx = tf.cast(example['Yx'], tf.int64)
    y = tf.io.decode_raw(example['Y'], tf.float32)
    y = tf.reshape(y, (yx,))
    return x, y

In [5]:
import itertools

In [6]:
def create_shards(filenames, shuffle = True):
    shards = tf.data.Dataset.from_tensor_slices(filenames[0])
    if shuffle == True:
        shards = shards.shuffle(len(filenames[0]))
    for i in range(1, len(filenames)):
        t_shards = tf.data.Dataset.from_tensor_slices(filenames[i])
        if shuffle == True:
            t_shards = t_shards.shuffle(len(filenames[i]))
        shards = shards.concatenate(t_shards)
    shards = shards.repeat()
    return shards

In [7]:
def get_dataset(filenames, model_type, shuffle, batch_size, buffer_size, workers):
    """Read TFRecords files and turn them into a TFRecordDataset."""
    shards = create_shards(filenames)
    if shuffle == True:
        dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length = 5, block_length = 1)
    else:
        dataset = shards.interleave(tf.data.TFRecordDataset, cycle_length = 1, block_length = 1)
    #   dataset = dataset.shuffle(buffer_size=8192)
    dataset = dataset.map(map_func = lambda x: decode(x, model_type), num_parallel_calls = workers).batch(batch_size = batch_size)
    dataset = dataset.prefetch(buffer_size = buffer_size)
    return dataset

In [8]:
def create_filenames(N_noise, shardsTVT = {"train": 320, "validation": 40, "test": 40}):
    filenames = {
        "train": [],
        "validation": [],
        "test": []
        }
    for key in filenames.keys():
        for seed in range(N_noise):
            filenames[key].append([inputs.data_fstring.format(key, seed, i, shardsTVT[key]-1) for i in range(shardsTVT[key])])
    return filenames

In [9]:
filenames = create_filenames(N_noise = 10)
train_ds = get_dataset(filenames["train"], ctx.inputs.model_type, shuffle = True, batch_size = 20, buffer_size = 16, workers = 3)
validation_ds = get_dataset([filenames["validation"][0]], ctx.inputs.model_type, shuffle = False, batch_size = 20, buffer_size = 16, workers = 3)

Tensor("args_0:0", shape=(), dtype=string)
Tensor("args_0:0", shape=(), dtype=string)


In [10]:
from src.py21cnn import utilities
from src.py21cnn.formatting import Filters

In [11]:
import hashlib
class LargeData:
    def __init__(
        self,
        dimensionality = 2,
        removed_average = True,
        normalized = True,
        Zmax = 30,
        filetype = 'float32',
        formatting = [],
        noise = ["tools21cm", "SKA1000"],
        shape = None,
        load_all = False,
        ):
        self.dimensionality = dimensionality
        self.removed_average = removed_average
        self.normalized = normalized
        self.Zmax = Zmax
        self.filetype = filetype
        if len(formatting) == 0:
            default_formatting = ['clipped_-250_+50', 'NaN_removed', 'TVT_parameterwise']
            if self.dimensionality == 2:
                default_formatting.append('boxcar44')
                default_formatting.append('10slices')
            if self.dimensionality == 3:
                default_formatting.append('boxcar444')
                default_formatting.append('sliced22')
            # default_formatting.sort()
            self.formatting = default_formatting
        else:
            # formatting.sort()
            self.formatting = formatting
        self.noise = noise + [f"walkers_{ctx.inputs.N_walker}", f"slices_{ctx.inputs.N_slice}", f"noise_{ctx.inputs.N_noise}"]
        self.shape = shape
        self.load_all = load_all
        self.load()

    def __str__(self):
        self.formatting.sort()
        self.noise.sort()
        S = f"dim:{self.dimensionality}__removed_average:{self.removed_average}__normalized:{self.normalized}__Zmax:{self.Zmax}__dtype:{self.filetype}"
        for i in self.formatting:
            S += f"__{i}"
        for i in self.noise:
            S += f"__{i}"
        return S
    
    def hash(self):
        return hashlib.md5(self.__str__().encode()).hexdigest()

    def load(self):
        permutation = Filters.constructIndexArray(ctx.inputs.N_walker, *ctx.inputs.pTVT, 1312)
        # print(permutation)
        self.partition = {
            "train": [], 
            "validation": [], 
            "test": []}
        self.noise_rolling_partition = {
            "train": [], 
            "validation": [], 
            "test": []}
        keys = list(self.partition.keys())
        for key in keys:
            for seed in range(ctx.inputs.N_noise):
                self.noise_rolling_partition[key].append([])
        self.inputs = {}
        for walker in range(ctx.inputs.N_walker):
            for s in range(ctx.inputs.N_slice):
                for seed in range(ctx.inputs.N_noise):
                    ID = ctx.inputs.X_fstring.format(walker, s, seed)
                    self.partition[keys[permutation[walker]]].append(ID)
                    self.noise_rolling_partition[keys[permutation[walker]]][seed].append(ID)
                    if self.load_all == True:
                        self.inputs[ID] = np.load(f"{ctx.inputs.data_location}{ID}.npy")

In [12]:
###############################################################################
#constructing TVT partitions of the data and assigning labels
###############################################################################
data_shape = ctx.inputs.X_shape[::-1] + (1,) if ctx.inputs.model_type == "RNN" else ctx.inputs.X_shape + (1,)
ctx.Data = LargeData(dimensionality = 3, shape = data_shape, load_all = False)
print(ctx.Data.hash())
ctx.filepath = f"{ctx.inputs.saving_location}{ctx.inputs.file_prefix}{ctx.inputs.model[0]}_{ctx.inputs.model[1]}_{ctx.HP.hash()}_{ctx.Data.hash()}"
ctx.logdir = f"{ctx.inputs.logs_location}{ctx.inputs.file_prefix}{ctx.inputs.model[0]}/{ctx.inputs.model[1]}/{ctx.Data.hash()}/{ctx.HP.hash()}"

b515f26321d231c6b06f7c086ce3e097


In [13]:
import importlib
ModelClassObject = getattr(importlib.import_module(f'src.py21cnn.architectures.{ctx.inputs.model[0]}'), ctx.inputs.model[1])
ModelClass = ModelClassObject(data_shape, HP)
ModelClass.build()

ctx.model = ModelClass.model

Model: "playground_SummarySpace3D_simple"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 526, 25, 25, 1)]  0         
_________________________________________________________________
conv1 (TimeDistributed)      (None, 526, 18, 18, 64)   4160      
_________________________________________________________________
maxpool (TimeDistributed)    (None, 526, 64)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 526, 64)           256       
_________________________________________________________________
dropout (TimeDistributed)    (None, 526, 64)           0         
_________________________________________________________________
cu_dnnlstm (CuDNNLSTM)       (None, 16)                5248      
_________________________________________________________________
dense (Dense)                (None

In [14]:
class Data:
    pass

In [15]:
import os
class TimeHistory(keras.callbacks.Callback):
    def __init__(self, filename):
        self.filename = filename

    def on_train_begin(self, logs={}):
        self.file = open(self.filename, 'a')

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.file.write(f"{time.time() - self.epoch_time_start}\n")
        self.file.flush()
        os.fsync(self.file.fileno())
    def on_train_end(self, logs={}):
        self.file.close()


class LR_tracer(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        lr = keras.backend.eval( self.model.optimizer.lr )
        print(f"LR: {lr:.10f}")


class LR_scheduler:
    def __init__(self, total_epochs, inital_LR, multi_gpu_run = False, reduce_factor = 0.1):
        self.total_epochs = total_epochs
        self.initial_LR = inital_LR
        self.multi_gpu_run = multi_gpu_run
        self.reduce_factor = reduce_factor
    def scheduler(self, epoch):
        """
        Returns learning rate at a given epoch. 
        Recieves total number of epochs and initial learning rate
        """
        # print(f"IN LR_scheduler, initLR {self.initial_LR}, epoch {epoch}, frac. {(epoch + 1) / self.total_epochs}")
        if (epoch + 1) / self.total_epochs < 0.5:
            return self.initial_LR
        elif (epoch + 1) / self.total_epochs < 0.75:
            return self.initial_LR * self.reduce_factor
        else:
            return self.initial_LR * self.reduce_factor ** 2
    def callback(self):
        if self.multi_gpu_run == True:
            return hvd.callbacks.LearningRateScheduleCallback(self.scheduler)
        else:
            return tf.keras.callbacks.LearningRateScheduler(self.scheduler)


def R2(y_true, y_pred):
        SS_res = keras.backend.sum(keras.backend.square(y_true-y_pred)) 
        SS_tot = keras.backend.sum(keras.backend.square(y_true - keras.backend.mean(y_true, axis=0)))
        return (1 - SS_res/(SS_tot + keras.backend.epsilon()))
def R2_numpy(y_true, y_pred):
        SS_res = np.sum((y_true - y_pred)**2) 
        SS_tot = np.sum((y_true - np.mean(y_true))**2)
        return (1 - SS_res/(SS_tot + 1e-7))
def R2_final(y_true, y_pred):
        SS_res = np.sum((y_true - y_pred)**2)
        SS_tot = np.sum((y_true - np.mean(y_true, axis=0))**2)
        return (1 - SS_res/(SS_tot + 1e-7))


def define_callbacks():
    if ctx.inputs.gpus == 1:
        saving_callbacks = True
        horovod_callbacks = False
    else:
        saving_callbacks = True if hvd.rank() == 0 else False
        horovod_callbacks = True

    if saving_callbacks == True:
        saving_callbacks = [
            # keras.callbacks.TensorBoard(ctx.logdir, update_freq="epoch"),
            # hp.KerasCallback(logdir, HP_TensorBoard),
            TimeHistory(f"{ctx.filepath}_time.txt"),
            keras.callbacks.ModelCheckpoint(f"{ctx.filepath}_best.hdf5", monitor='val_loss', save_best_only=True, verbose=True),
            keras.callbacks.ModelCheckpoint(f"{ctx.filepath}_last.hdf5", monitor='val_loss', save_best_only=False, verbose=True), 
            keras.callbacks.CSVLogger(f"{ctx.filepath}.log", separator=',', append=True),
            LR_tracer(),
            ]
    else:
        saving_callbacks = []
    if horovod_callbacks == True:
        horovod_callbacks = [
            hvd.callbacks.BroadcastGlobalVariablesCallback(0),
            hvd.callbacks.MetricAverageCallback(),
            ]
        if ctx.load_model == False:
            horovod_callbacks.append(hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=ctx.inputs.warmup))
    else:
        horovod_callbacks = []
    
    important_callbacks = [
        keras.callbacks.TerminateOnNaN(),
        ]
    if ctx.HP.ReducingLR == True:
        scheduler = LR_scheduler(ctx.HP.MaxEpochs, ctx.HP.LearningRate, reduce_factor = 0.1)
        important_callbacks.append(scheduler.callback())

    #not saving into ctx as it might screw up things during broadcast of variables
    return horovod_callbacks + saving_callbacks + important_callbacks


def define_model(restore_training):
    model_exists = os.path.exists(f"{ctx.filepath}_last.hdf5")
    #define in what case to load the model
    # if model_exists == True and restore_training == True:
    #     if ctx.inputs.gpus == 1:
    #         load_model = True
    #         load_function = keras.models.load_model
    #     else:
    #         if hvd.rank() == 0:
    #             load_model = True
    #             load_function = hvd.load_model
    #         else:
    #             load_model = False
    # else:
    #     load_model = False
    if model_exists == True and restore_training == True:
        load_model = True
        if ctx.inputs.gpus == 1:
            load_function = keras.models.load_model
        else:
            load_function = hvd.load_model
    else:
        load_model = False
    ctx.load_model = load_model

    #determine steps_per_epoch
    if isinstance(ctx.Data, Data):
        steps_per_epoch = ctx.Data.X["train"].shape[0] // ctx.inputs.gpus // ctx.HP.BatchSize
        validation_steps = ctx.Data.X["val"].shape[0] // ctx.HP.BatchSize
    elif isinstance(ctx.Data, LargeData):
        if ctx.inputs.noise_rolling == True:
            steps_per_epoch = len(ctx.Data.noise_rolling_partition["train"][0]) // ctx.inputs.gpus // ctx.HP.BatchSize
            validation_steps = len(ctx.Data.noise_rolling_partition["validation"][0]) // ctx.HP.BatchSize   
        else:         
            steps_per_epoch = len(ctx.Data.partition["train"]) // ctx.inputs.gpus // ctx.HP.BatchSize
            validation_steps = len(ctx.Data.partition["validation"]) // ctx.HP.BatchSize
    else:
        raise TypeError("ctx.Data should be an instance of {Data, LargeData} class")

    if validation_steps == 0:
        raise ValueError("Number of validation steps per epoch is 0. Change batch size, validation probability, or give me more data.")

    #load the model
    if load_model == True:
        custom_obj = {}
        custom_obj["R2"] = R2
        if ctx.HP.ActivationFunction[0] == "leakyrelu":
            custom_obj[ctx.HP.ActivationFunction[0]] = ctx.HP.ActivationFunction[1]["activation"]
        #if loading last model fails for some reason, load the best one
        try:
            ctx.model = load_function(f"{ctx.filepath}_last.hdf5", custom_objects=custom_obj)
        except:
            ctx.model = load_function(f"{ctx.filepath}_best.hdf5", custom_objects=custom_obj)

        with open(f"{ctx.filepath}.log") as f:
            number_of_epochs_trained = len(f.readlines()) - 1  #the first line is description
            print("NUMBER_OF_EPOCHS_TRAINED", number_of_epochs_trained)
        if ctx.HP.Epochs + number_of_epochs_trained > ctx.HP.MaxEpochs:
            final_epochs = ctx.HP.MaxEpochs
        else:
            final_epochs = ctx.HP.Epochs + number_of_epochs_trained

        ctx.fit_options = {
            "epochs": final_epochs,
            "initial_epoch": number_of_epochs_trained,
            "steps_per_epoch": steps_per_epoch,
            "validation_steps": validation_steps,
            }
        ctx.compile_options = {}
    else:
        ctx.fit_options = {
            "epochs": ctx.HP.Epochs,
            "initial_epoch": 0,
            "steps_per_epoch": steps_per_epoch,
            "validation_steps": validation_steps,
            }
        ctx.compile_options = {
            "loss": ctx.HP.Loss[1],
            "optimizer": ctx.HP.Optimizer[0](**ctx.HP.Optimizer[2]),
            "metrics": [R2],
            }
        if ctx.inputs.gpus > 1:
            ctx.compile_options["optimizer"] = hvd.DistributedOptimizer(ctx.compile_options["optimizer"])


callbacks = define_callbacks()
define_model(restore_training = False)
if len(ctx.compile_options) > 0:
    ctx.model.compile(**ctx.compile_options)

In [16]:
print(ctx.compile_options)
print(ctx.fit_options)

{'loss': 'mse', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7fe480a18438>, 'metrics': [<function R2 at 0x7fe30c6379d8>]}
{'epochs': 10, 'initial_epoch': 0, 'steps_per_epoch': 16000, 'validation_steps': 2000}


In [17]:
import time

In [18]:
history = ctx.model.fit(
    train_ds,
    validation_data = validation_ds,
    callbacks = callbacks,
    epochs = 3,
    steps_per_epoch = 20,
    validation_steps = 10,
    )

Train for 20 steps, validate for 10 steps
LR: 0.0010000000
Epoch 1/3
19/20 [===========================>..] - ETA: 0s - loss: 1.0453 - R2: -0.1192
Epoch 00001: val_loss improved from inf to 1.03812, saving model to /scratch/d.prelogovic/runs/models/playground_SummarySpace3D_simple_c499ab7d84c5b92a5928da1a728918d3_b515f26321d231c6b06f7c086ce3e097_best.hdf5

Epoch 00001: saving model to /scratch/d.prelogovic/runs/models/playground_SummarySpace3D_simple_c499ab7d84c5b92a5928da1a728918d3_b515f26321d231c6b06f7c086ce3e097_last.hdf5
20/20 [==============================] - 7s 348ms/step - loss: 1.0539 - R2: -0.1196 - val_loss: 1.0381 - val_R2: -0.0676
LR: 0.0010000000
Epoch 2/3
19/20 [===========================>..] - ETA: 0s - loss: 1.0376 - R2: -0.0853
Epoch 00002: val_loss improved from 1.03812 to 1.02451, saving model to /scratch/d.prelogovic/runs/models/playground_SummarySpace3D_simple_c499ab7d84c5b92a5928da1a728918d3_b515f26321d231c6b06f7c086ce3e097_best.hdf5

Epoch 00002: saving model t

In [19]:
history.params


{'batch_size': None,
 'epochs': 3,
 'steps': 20,
 'samples': 20,
 'verbose': 0,
 'do_validation': True,
 'metrics': ['loss', 'R2', 'val_loss', 'val_R2']}